In [1]:
import pandas as pd

df = pd.read_csv("Results.csv")

df

,Text_ID,Project,Free_Text_KEY,Free_Text_title,Text,Is_latest_text
0,3,IDN0174,5,Project Objective,To recover the social and economic activity in...,0
1,12843,TUN1004,5,Project Objective,the objective is to address the urgent needs o...,1
2,12868,JOR1001,5,Project Objective,The objective of this project is to promote re...,1
3,12870,BHR1002,5,Project Objective,The objective of the project is to promote you...,1
4,12871,YEM1001,5,Project Objective,The objective of the Program is to contribute ...,1
...,...,...,...,...,...,...
27208,185127,PAK1060,459,Readiness,The project is part of the Work Program for 20...,1
27209,185129,MOZ1022,5,Project Objective,To organize cataract screening campaigns and p...,1
27210,185130,MOZ1022,7,Scope,"To perform screening 10,000 patients and Condu...",1
27211,185131,MOZ1022,6,Project Locations,Mozabique,1


In [43]:
selected_fields = ['Scope' , 'Project Relevance' , 'Project Context' , 'Project Objective' , 'Project Rationale']

#selected_fields = [ 'Project Objective' ]

filtered_df = df[df['Free_Text_title'].isin(selected_fields) & df['Is_latest_text'] == 1 ]

df2 = filtered_df[['Project' , 'Free_Text_title' , 'Text']].pivot(index = ['Project'] , columns='Free_Text_title' , values = 'Text').dropna()

df2.reset_index(inplace=True)

In [13]:
pd.unique(df2.Free_Text_title)

array(['Project Objective', 'Scope', 'Project Rationale',
       'Previous Experience with Executing Agency (Yes/No)',
       'Project Locations', 'Executing Agency',
       'Status of Project Preparation: By IDB', 'Recommendations',
       'Lessons Learnt', 'Issues/Action Taken',
       'Status of Project Preparation: By Beneficiary', 'Remarks',
       'Project Relevance', 'Project Context', 'Project Risks',
       'Country Ownership',
       'Availability and Quality of project documents', 'Project Results',
       'Strategic Alignment of Project', 'Project Current Status'],
      dtype=object)

In [52]:
df_in = df2[['Project' , 'Project Objective']]


In [56]:
from langchain.document_loaders import DataFrameLoader

loader = DataFrameLoader(df_in, page_content_column="Project Objective")

docs = loader.load()

In [57]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [58]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

store = Chroma.from_documents(docs, embeddings)



In [60]:
store.similarity_search("Which projects eleviate poverty?")

[Document(page_content='Support vulnerable households in rural communities to have improved food security, household income, and resilience to climate change through the construction of agro-based infrastructure projects in the provinces of Badakhshan, Faryab, and Ghazni.', metadata={'Project': 'AFG1037'}),
 Document(page_content='The overall goal of this pilot intervention is to support vulnerable livelihoods and reduce food insecurity in vulnerable communities in southern Afghanistan.\nThe project is also expected to support the reduction of illicit crops cultivation, while, at the same time, promoting the economic empowerment of women.', metadata={'Project': 'AFG1027'}),
 Document(page_content='Improve food security of vulnerable households through diversified Livelihood opportunities and Increase Agriculture and Livestock Production as main sources of livelihood for targeted communities.', metadata={'Project': 'AFG1032'}),
 Document(page_content=' Project Development Objective is a

In [61]:
from langchain.llms import OpenAIChat

from langchain.chains import RetrievalQA

retriever = store.as_retriever()

llm = OpenAIChat(temperature = 0.0, model_name="gpt-3.5-turbo-16k-0613")

qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

/Users/altaf/Library/Python/3.9/lib/python/site-packages/langchain/llms/openai.py:801: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [65]:
query = "Provide a short summary on the projects related to housing"

res = qa_chain.run(query)

print(res)



> Entering new RetrievalQA chain...

> Finished chain.
The projects related to housing aim to expand access to affordable finance for constructing sustainable multi-storied housings with improved quality and necessary basic facilities for lower and middle-income people dwelling in rural and peri-urban areas. These projects also address climate change by introducing climate-resilient and eco-friendly housing.
